# Completion (02nd October 2023)

- Iterate by date (Some surveys have the same date)

# Clarifications


# Questions to Ask


In [ ]:
from utils import *

from daily_calculator import DailyCalculator

import pandas as pd

import nest_asyncio
import asyncio

# Only run nest_asyncio in a Jupyter Notebook environment
nest_asyncio.apply()

In [ ]:
# Load the respective files

async def get_all_dataframes():
    return await asyncio.gather(
        load_intake24(),
        load_heifa_ingredients(),
        load_heifa_recipes()
    )

intake24_df, heifa_food_df, heifa_recipes_df = asyncio.run(get_all_dataframes())

In [ ]:
intake24_df.query("user_id == 195")[['meal_name', 'heifa_nutrient_id']].head(30)

# Breakdown of Intake 24:

The file has many users.

Each user has many surveys.

Each survey has many meal intake.

Each intake consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [ ]:
user_dict = create_user_objects(intake24_df)

for user_id in user_dict.keys():

    print(f"Printing for User {user_id}")
    user_obj = user_dict[user_id]
    user_obj.print_information()

# Breakdown of HEIFA (Food Composition)

Every row in the file is a unique ingredient.

Every ingredient:
- has it's own attributes.
- can be mapped to a 8-digit code (for HEIFA Recipe)
- is used as a divisor for either energy (kilo joules) or grams (g)

In [ ]:
# Create the objects
food_composition_dict = create_food_objects(heifa_food_df)

for key, food_comp_obj in food_composition_dict.items():

    food_comp_obj.print_full_details()

# Breakdown of HEIFA (Recipes)

- Every recipe has multiple ingredients
- Keys are repeated across rows (similar to Survey ID of Intake24)
- Every ingredient has respective proportion to the recipe

In [ ]:
recipe_dict = create_recipe_objects(heifa_recipes_df)

for id, recipe_obj in recipe_dict.items():
    print(f"Printing for ID {id}\n")
    recipe_obj.print_ingredients_information()

## Mapping between Intake24 and HEIFA Ingredients

- For each user, extract the given nutrients and store in an array.
- This is from ALL the survey data.
- We don't care about the order here.
- The array will contain a list of dictionaries/JSON.

In the array:

- Use the HEIFA ID (from user) to map to the HEIFA Ingredients' HEIFA ID.
- Check if a result is found or not.
- Check if it requires a recipe or not.

## Mapping between Intake24 and HEIFA Recipes

This is in case a recipe is found (The second step).

- For the given recipe, extract the given nutrients ID and proportion, store in an array.
- We don't care about the order here.
- The array will contain a list of dictionaries.

In the array:

- Use the HEIFA ID (from the recipes) to map the HEIFA Ingredients' HEIFA ID.
- Check the energy and serving size.

In [ ]:
# TODO:
# - Create serving calculator object
# - Calculate daily basis
# - Store in this hierarchy: Date -> Food group -> Serving size
# - Save this in the user object under daily_servings

In [ ]:
user_meals = fetch_user_food_list(user_dict)

for user_id, meal_date_dict in user_meals.items():

    print(f"Printing for user {user_id}")

    calculate_portion_serving_heifa(meal_date_dict, food_composition_dict, recipe_dict)
    print("&" * 20)
    print("\n\n")

In [ ]:
user_meals = fetch_user_food_list(user_dict)
daily_calculator = DailyCalculator(food_composition_dict, recipe_dict)

user_daily_intake = {}

user_daily_intake = {
    user_id: daily_calculator.calculate_daily_servings(meal_date_dict)
    for user_id, meal_date_dict in user_meals.items()
}

In [ ]:


# Display

for user_id, daily_intake_dict in user_daily_intake.items():

    print(f"Printing for user {user_id}\n")

    for date, food_group_dict in daily_intake_dict.items():

        print(f"For date {date}\n")
        print("*" * 20)
        for food_group, total_serving in food_group_dict.items():
            print(f"{food_group}: {total_serving:.2f} serves")
        
        
        print("*" * 20)
        print("\n")
    
    print("=" * 20)
    print("\n")